In [ ]:

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import os


data_dir = "train"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
class_names = dataset.classes


model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


model.train()
for epoch in range(3):
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


torch.save(model.state_dict(), "skin_model.pth")
with open("class_names.txt", "w") as f:
    for name in class_names:
        f.write(name + "\n")


Epoch 1, Loss: 1.0893
Epoch 2, Loss: 0.5965
Epoch 3, Loss: 0.3874


In [ ]:

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn


val_dir = "val"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(val_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16)

with open("class_names.txt") as f:
    class_names = [line.strip() for line in f]


model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load("skin_model.pth"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {100 * correct / total:.2f}%")


C:\Users\shuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Accuracy: 85.64%


In [ ]:

import torch
from torchvision import transforms, models
from PIL import Image
import sys


templates = {
    "Melanoma": "The image indicates Melanoma, a serious form of skin cancer. Immediate dermatological evaluation is advised.",
    "Actinic keratosis": "Features are consistent with Actinic Keratosis, a precancerous skin lesion caused by sun exposure.",
    "Benign keratosis": "This appears to be Benign Keratosis, which is typically non-cancerous and harmless.",
    "Dermatofibroma": "The lesion resembles a Dermatofibroma, a common benign skin nodule.",
    "Melanocytic nevus": "The image shows a Melanocytic Nevus, a type of mole, usually benign.",
    "Squamous cell carcinoma": "Signs point to Squamous Cell Carcinoma, a type of skin cancer requiring medical treatment.",
    "Tinea Ringworm Candidiasis": "The image resembles a fungal infection like Tinea or Candidiasis, often treatable with antifungal agents.",
    "Atopic Dermatitis": "The features are typical of Atopic Dermatitis, a chronic inflammatory skin condition.",
    "Vascular lesion": "Indications of a Vascular Lesion, usually benign, but may need further evaluation depending on symptoms.",
}


with open("class_names.txt") as f:
    class_names = [line.strip() for line in f]


model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load("skin_model.pth"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output, 1).item()
        class_name = class_names[pred]
        report = templates.get(class_name, "No report available.")
        return class_name, report


image_path = "predict/melanoma.jpg"  
disease, report = predict(image_path)
print("Predicted Class:", disease)
print("Generated Report:", report)

C:\Users\shuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\shuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Predicted Class: Melanoma
Generated Report: The image indicates Melanoma, a serious form of skin cancer. Immediate dermatological evaluation is advised.
